In [11]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch_geometric
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# ... (Keep the data loading and preprocessing code the same)
# Load the data
df = pd.read_csv('final_data.csv')
df['datetime'] = pd.to_datetime(df['datetime'], format='%d-%m-%Y %H:%M')
df.set_index('datetime', inplace=True)

# Feature engineering
df['hour'] = df.index.hour
df['day_of_week'] = df.index.dayofweek
df['month'] = df.index.month
df['is_weekend'] = df['day_of_week'].isin([5, 6]).astype(int)

# Create lag features
for target in ['DELHI', 'BRPL', 'BYPL', 'NDMC', 'MES']:
    df[f'{target}_lag_1'] = df[target].shift(1)
    df[f'{target}_lag_24'] = df[target].shift(24)

# Drop rows with NaN values after creating lag features
df.dropna(inplace=True)

# Prepare features and target variables
features = ['temperature', 'humidity', 'wind_speed', 'precipitation', 'hour', 'day_of_week', 'month', 'is_weekend'] + \
           [f'{target}_lag_1' for target in ['DELHI', 'BRPL', 'BYPL', 'NDMC', 'MES']] + \
           [f'{target}_lag_24' for target in ['DELHI', 'BRPL', 'BYPL', 'NDMC', 'MES']]
targets = ['DELHI', 'BRPL', 'BYPL', 'NDMC', 'MES']

# Normalize features and targets
scaler_X = StandardScaler()
X = scaler_X.fit_transform(df[features])

scalers_y = {}
y = np.zeros((len(df), len(targets)))
for i, target in enumerate(targets):
    scaler = StandardScaler()
    y[:, i] = scaler.fit_transform(df[[target]]).ravel()
    scalers_y[target] = scaler

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Print shapes for debugging
print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")
# Modify the create_graph_data function
def create_graph_data(X, y, window_size=24, stride=1):
    graphs = []
    for i in range(0, len(X) - window_size + 1, stride):
        x = torch.FloatTensor(X[i:i+window_size])
        y_sample = torch.FloatTensor(y[i+window_size-1])
        
        edge_index = torch.tensor(np.array(np.meshgrid(np.arange(window_size), np.arange(window_size))).reshape(2, -1), dtype=torch.long)
        
        graphs.append(Data(x=x, edge_index=edge_index, y=y_sample))
    return graphs

# ... (Keep the data preparation code the same)

# Modify the Hybrid GNN-GRU model
class HybridGNNGRU(nn.Module):
    def __init__(self, input_dim, hidden_dim, gru_hidden_dim, output_dim):
        super(HybridGNNGRU, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.gru = nn.GRU(hidden_dim, gru_hidden_dim, batch_first=True)
        self.linear = nn.Linear(gru_hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        
        x = torch.relu(self.conv1(x, edge_index))
        x = torch.relu(self.conv2(x, edge_index))
        
        x = global_mean_pool(x, batch)
        x = x.view(-1, 1, x.size(1))  # (batch_size, seq_len=1, features)
        
        _, h_n = self.gru(x)
        x = h_n.squeeze(0)
        
        x = self.linear(x)
        return x

# Initialize model, loss function, and optimizer
model = HybridGNNGRU(input_dim=X.shape[1], hidden_dim=64, gru_hidden_dim=32, output_dim=len(targets))
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Modify the training loop
def train():
    model.train()
    total_loss = 0
    for data in train_loader:
        optimizer.zero_grad()
        out = model(data)
        # Reshape the target tensor to match the model output
        target = data.y.view(-1, len(targets))
        loss = criterion(out, target)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * data.num_graphs
    return total_loss / len(train_loader.dataset)

# Modify the evaluation function
def test(loader):
    model.eval()
    total_loss = 0
    predictions = []
    actuals = []
    with torch.no_grad():
        for data in loader:
            out = model(data)
            # Reshape the target tensor to match the model output
            target = data.y.view(-1, len(targets))
            loss = criterion(out, target)
            total_loss += loss.item() * data.num_graphs
            predictions.append(out.cpu().numpy())
            actuals.append(target.cpu().numpy())
    predictions = np.concatenate(predictions)
    actuals = np.concatenate(actuals)
    return total_loss / len(loader.dataset), predictions, actuals

# Train the model
num_epochs = 100
for epoch in range(num_epochs):
    loss = train()
    test_loss, _, _ = test(test_loader)
    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {loss:.4f}, Test Loss: {test_loss:.4f}')

# Final evaluation
_, y_pred, y_true = test(test_loader)

# Inverse transform predictions and actual values
y_true_inv = np.zeros_like(y_true)
y_pred_inv = np.zeros_like(y_pred)
for i, target in enumerate(targets):
    y_true_inv[:, i] = scalers_y[target].inverse_transform(y_true[:, i].reshape(-1, 1)).ravel()
    y_pred_inv[:, i] = scalers_y[target].inverse_transform(y_pred[:, i].reshape(-1, 1)).ravel()

# Calculate metrics
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

for i, target in enumerate(targets):
    rmse = np.sqrt(mean_squared_error(y_true_inv[:, i], y_pred_inv[:, i]))
    mape = mean_absolute_percentage_error(y_true_inv[:, i], y_pred_inv[:, i])
    r2 = r2_score(y_true_inv[:, i], y_pred_inv[:, i])
    
    print(f"{target}:")
    print(f"  RMSE: {rmse:.2f}")
    print(f"  MAPE: {mape:.2f}%")
    print(f"  R2: {r2:.2f}")
    print()

X shape: (11011, 18)
y shape: (11011, 5)
Epoch 1/100, Train Loss: 0.9782, Test Loss: 0.9472
Epoch 2/100, Train Loss: 0.9690, Test Loss: 0.9443
Epoch 3/100, Train Loss: 0.9679, Test Loss: 0.9444
Epoch 4/100, Train Loss: 0.9655, Test Loss: 0.9457
Epoch 5/100, Train Loss: 0.9657, Test Loss: 0.9442
Epoch 6/100, Train Loss: 0.9632, Test Loss: 0.9427
Epoch 7/100, Train Loss: 0.9631, Test Loss: 0.9423
Epoch 8/100, Train Loss: 0.9617, Test Loss: 0.9448
Epoch 9/100, Train Loss: 0.9601, Test Loss: 0.9443
Epoch 10/100, Train Loss: 0.9591, Test Loss: 0.9446
Epoch 11/100, Train Loss: 0.9580, Test Loss: 0.9456
Epoch 12/100, Train Loss: 0.9567, Test Loss: 0.9523
Epoch 13/100, Train Loss: 0.9549, Test Loss: 0.9478
Epoch 14/100, Train Loss: 0.9544, Test Loss: 0.9485
Epoch 15/100, Train Loss: 0.9511, Test Loss: 0.9508
Epoch 16/100, Train Loss: 0.9492, Test Loss: 0.9488
Epoch 17/100, Train Loss: 0.9468, Test Loss: 0.9526
Epoch 18/100, Train Loss: 0.9436, Test Loss: 0.9611
Epoch 19/100, Train Loss: 0.9431

In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch_geometric
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# Load the data
df = pd.read_csv('final_data.csv')
df['datetime'] = pd.to_datetime(df['datetime'], format='%d-%m-%Y %H:%M')
df.set_index('datetime', inplace=True)

# Feature engineering
df['hour'] = df.index.hour
df['day_of_week'] = df.index.dayofweek
df['month'] = df.index.month
df['is_weekend'] = df['day_of_week'].isin([5, 6]).astype(int)

# Create lag features
for target in ['DELHI', 'BRPL', 'BYPL', 'NDMC', 'MES']:
    df[f'{target}_lag_1'] = df[target].shift(1)
    df[f'{target}_lag_24'] = df[target].shift(24)

# Drop rows with NaN values after creating lag features
df.dropna(inplace=True)

# Prepare features and target variables
features = ['temperature', 'humidity', 'wind_speed', 'precipitation', 'hour', 'day_of_week', 'month', 'is_weekend'] + \
           [f'{target}_lag_1' for target in ['DELHI', 'BRPL', 'BYPL', 'NDMC', 'MES']] + \
           [f'{target}_lag_24' for target in ['DELHI', 'BRPL', 'BYPL', 'NDMC', 'MES']]
targets = ['DELHI', 'BRPL', 'BYPL', 'NDMC', 'MES']

# Normalize features
scaler_X = StandardScaler()
X = scaler_X.fit_transform(df[features])

# Normalize target variables
scaler_y = StandardScaler()
y = scaler_y.fit_transform(df[targets])

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create graph data
def create_graph_data(X, y, window_size=24, stride=1):
    graphs = []
    for i in range(0, len(X) - window_size + 1, stride):
        x = torch.FloatTensor(X[i:i+window_size])
        y_sample = torch.FloatTensor(y[i+window_size-1])
        
        edge_index = torch.tensor(np.array(np.meshgrid(np.arange(window_size), np.arange(window_size))).reshape(2, -1), dtype=torch.long)
        
        graphs.append(Data(x=x, edge_index=edge_index, y=y_sample))
    return graphs

train_graphs = create_graph_data(X_train, y_train)
test_graphs = create_graph_data(X_test, y_test)

train_loader = DataLoader(train_graphs, batch_size=32, shuffle=True)
test_loader = DataLoader(test_graphs, batch_size=32, shuffle=False)

# Define Hybrid GNN-GRU model
class HybridGNNGRU(nn.Module):
    def __init__(self, input_dim, hidden_dim, gru_hidden_dim, output_dim, num_layers=3):
        super(HybridGNNGRU, self).__init__()
        self.conv_layers = nn.ModuleList([GCNConv(input_dim if i == 0 else hidden_dim, hidden_dim) for i in range(num_layers)])
        self.gru = nn.GRU(hidden_dim, gru_hidden_dim, num_layers=2, batch_first=True, bidirectional=True)
        self.linear1 = nn.Linear(gru_hidden_dim * 2, gru_hidden_dim)
        self.linear2 = nn.Linear(gru_hidden_dim, output_dim)
        self.dropout = nn.Dropout(0.2)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        
        for conv in self.conv_layers:
            x = torch.relu(conv(x, edge_index))
            x = self.dropout(x)
        
        x = global_mean_pool(x, batch)
        x = x.view(-1, 1, x.size(1))  # (batch_size, seq_len=1, features)
        
        _, h_n = self.gru(x)
        x = torch.cat((h_n[-2,:,:], h_n[-1,:,:]), dim=1)
        x = torch.relu(self.linear1(x))
        x = self.dropout(x)
        x = self.linear2(x)
        return x

# Initialize model, loss function, and optimizer
model = HybridGNNGRU(input_dim=X.shape[1], hidden_dim=128, gru_hidden_dim=64, output_dim=len(targets))
criterion = nn.HuberLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=10, verbose=True)

# Training function
def train():
    model.train()
    total_loss = 0
    for data in train_loader:
        optimizer.zero_grad()
        out = model(data)
        target = data.y.view(-1, len(targets))
        loss = criterion(out, target)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * data.num_graphs
    return total_loss / len(train_loader.dataset)

# Evaluation function
def test(loader):
    model.eval()
    total_loss = 0
    predictions = []
    actuals = []
    with torch.no_grad():
        for data in loader:
            out = model(data)
            target = data.y.view(-1, len(targets))
            loss = criterion(out, target)
            total_loss += loss.item() * data.num_graphs
            predictions.append(out.cpu().numpy())
            actuals.append(target.cpu().numpy())
    predictions = np.concatenate(predictions)
    actuals = np.concatenate(actuals)
    return total_loss / len(loader.dataset), predictions, actuals

# Train the model
num_epochs = 100
best_test_loss = float('inf')
for epoch in range(num_epochs):
    loss = train()
    test_loss, _, _ = test(test_loader)
    scheduler.step(test_loss)
    if test_loss < best_test_loss:
        best_test_loss = test_loss
        torch.save(model.state_dict(), 'best_model.pth')
    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {loss:.4f}, Test Loss: {test_loss:.4f}')

# Load best model and evaluate
model.load_state_dict(torch.load('best_model.pth'))
_, y_pred, y_true = test(test_loader)

# Inverse transform predictions and actual values
y_true_inv = scaler_y.inverse_transform(y_true)
y_pred_inv = scaler_y.inverse_transform(y_pred)

# Calculate metrics
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / (y_true + 1e-8))) * 100  # Added small constant to avoid division by zero

for i, target in enumerate(targets):
    rmse = np.sqrt(mean_squared_error(y_true_inv[:, i], y_pred_inv[:, i]))
    mape = mean_absolute_percentage_error(y_true_inv[:, i], y_pred_inv[:, i])
    r2 = r2_score(y_true_inv[:, i], y_pred_inv[:, i])
    
    print(f"{target}:")
    print(f"  RMSE: {rmse:.2f}")
    print(f"  MAPE: {mape:.2f}%")
    print(f"  R2: {r2:.2f}")
    print()
    

C:\Users\Anuska Palit\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\optim\lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 1/100, Train Loss: 0.4239, Test Loss: 0.4097
Epoch 2/100, Train Loss: 0.4209, Test Loss: 0.4100
Epoch 3/100, Train Loss: 0.4204, Test Loss: 0.4102
Epoch 4/100, Train Loss: 0.4195, Test Loss: 0.4106
Epoch 5/100, Train Loss: 0.4200, Test Loss: 0.4092
Epoch 6/100, Train Loss: 0.4192, Test Loss: 0.4096
Epoch 7/100, Train Loss: 0.4186, Test Loss: 0.4095
Epoch 8/100, Train Loss: 0.4179, Test Loss: 0.4101
Epoch 9/100, Train Loss: 0.4168, Test Loss: 0.4105
Epoch 10/100, Train Loss: 0.4163, Test Loss: 0.4135
Epoch 11/100, Train Loss: 0.4143, Test Loss: 0.4135
Epoch 12/100, Train Loss: 0.4132, Test Loss: 0.4117
Epoch 13/100, Train Loss: 0.4110, Test Loss: 0.4136
Epoch 14/100, Train Loss: 0.4093, Test Loss: 0.4144
Epoch 15/100, Train Loss: 0.4065, Test Loss: 0.4196
Epoch 16/100, Train Loss: 0.4047, Test Loss: 0.4174
Epoch 17/100, Train Loss: 0.3940, Test Loss: 0.4259
Epoch 18/100, Train Loss: 0.3897, Test Loss: 0.4311
Epoch 19/100, Train Loss: 0.3856, Test Loss: 0.4340
Epoch 20/100, Train L

C:\Users\Anuska Palit\AppData\Local\Temp\ipykernel_200\394487467.py:145: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_model.pth'))


DELHI:
  RMSE: 1315.37
  MAPE: 31.54%
  R2: 0.04

BRPL:
  RMSE: 589.24
  MAPE: 33.06%
  R2: 0.03

BYPL:
  RMSE: 307.45
  MAPE: 36.53%
  R2: 0.04

NDMC:
  RMSE: 63.56
  MAPE: 29.91%
  R2: 0.02

MES:
  RMSE: 8.66
  MAPE: 5296208.98%
  R2: 0.02



In [2]:
y_true_inv 

array([[4171.86    , 1809.61    ,  948.2     ,  178.5     ,   25.3     ],
       [4719.78    , 1943.08    ,  994.71    ,  258.88    ,   33.489998],
       [2584.4502  , 1057.48    ,  538.38    ,  101.26    ,   19.79    ],
       ...,
       [4970.9     , 2052.77    , 1047.69    ,  289.8     ,   35.95    ],
       [3663.03    , 1344.42    ,  785.12    ,  129.57    ,   19.12    ],
       [6321.5503  , 2602.41    , 1359.33    ,  337.17    ,   42.85    ]],
      dtype=float32)

In [3]:
y_pred_inv

array([[4301.5815  , 1816.4857  ,  932.2737  ,  175.53427 ,   29.047318],
       [4364.547   , 1842.0323  ,  946.802   ,  177.76193 ,   29.350107],
       [4215.89    , 1779.2046  ,  912.55426 ,  172.74533 ,   28.583992],
       ...,
       [4133.171   , 1741.272   ,  893.24243 ,  169.72441 ,   28.087063],
       [3998.792   , 1686.696   ,  861.8116  ,  165.35356 ,   27.370144],
       [4180.762   , 1763.0199  ,  904.81036 ,  171.64081 ,   28.385561]],
      dtype=float32)